In [6]:
import numpy as np

def cargar_instancia_catspp(path_archivo):
    """
    Carga una instancia CATSPP en formato TSPLIB, reemplazando -1 por penalización fija de 1000000.
    
    Args:
        path_archivo (str): Ruta al archivo .tsp
    
    Returns:
        np.ndarray: Matriz de costos procesada
    """
    matriz = []
    leyendo_matriz = False
    dimension = None
    
    with open(path_archivo, 'r') as f:
        for linea in f:
            linea = linea.strip()
            if linea.startswith("DIMENSION"):
                dimension = int(linea.split()[-1])
            if linea == "EDGE_WEIGHT_SECTION":
                leyendo_matriz = True
                continue
            if linea == "EOF":
                break
            if leyendo_matriz:
                fila = [float(x) for x in linea.split()]
                matriz.append(fila)
    
    matriz = np.array(matriz)
    
    # Validación de dimensión
    if dimension is not None and matriz.shape[0] > dimension:
        matriz = matriz[:dimension, :dimension]
    
    # Penalización fija
    matriz[matriz == -1] = 999999
    
    return matriz


In [7]:
import random

def calcular_costo(matriz_costos, secuencia):
    """Calcula el costo total de la secuencia. Retorna inf si pasa por arco prohibido."""
    costo = 0
    for i in range(len(secuencia) - 1):
        c = matriz_costos[secuencia[i], secuencia[i + 1]]
        if c >= 999999:
            return float('inf')  # Solución no factible
        costo += c
    return costo

def greedy_catspp(matriz_costos, inicio=0):
    """
    Heurística greedy: elige siempre el siguiente nodo con menor costo permitido.
    
    Args:
        matriz_costos (np.ndarray): Matriz de costos
        inicio (int): Nodo inicial
    
    Returns:
        tuple: (secuencia, costo total)
    """
    n = matriz_costos.shape[0]
    visitados = set([inicio])
    secuencia = [inicio]
    costo_total = 0
    actual = inicio

    for _ in range(n - 1):
        candidatos = [(j, matriz_costos[actual, j]) for j in range(n) if j not in visitados]
        if not candidatos:
            break
        siguiente, costo = min(candidatos, key=lambda x: x[1])
        secuencia.append(siguiente)
        costo_total += costo
        visitados.add(siguiente)
        actual = siguiente
    
    return secuencia, costo_total

def two_opt_swap(secuencia, i, k):
    """Realiza un swap 2-opt entre los índices i y k."""
    nueva = secuencia[:i] + secuencia[i:k+1][::-1] + secuencia[k+1:]
    return nueva

def neighbourhood_search(matriz_costos, secuencia_inicial, max_iter=100):
    """
    Búsqueda local con 2-opt.
    
    Args:
        matriz_costos (np.ndarray): Matriz de costos
        secuencia_inicial (list): Secuencia inicial
        max_iter (int): Máximo de iteraciones
    
    Returns:
        tuple: (mejor_secuencia, mejor_costo)
    """
    n = len(secuencia_inicial)
    mejor_secuencia = secuencia_inicial[:]
    mejor_costo = calcular_costo(matriz_costos, mejor_secuencia)
    
    for _ in range(max_iter):
        mejora = False
        for i in range(1, n - 2):
            for k in range(i + 1, n - 1):
                nueva = two_opt_swap(mejor_secuencia, i, k)
                costo_nuevo = calcular_costo(matriz_costos, nueva)
                if costo_nuevo < mejor_costo:
                    mejor_secuencia = nueva
                    mejor_costo = costo_nuevo
                    mejora = True
                    break
            if mejora:
                break
        if not mejora:
            break
    
    return mejor_secuencia, mejor_costo

def grasp_catspp(matriz_costos, n_iter=50, alpha=0.3):
    """
    GRASP: Construcción aleatoria y búsqueda local.
    
    Args:
        matriz_costos (np.ndarray): Matriz de costos
        n_iter (int): Número de iteraciones
        alpha (float): Nivel de aleatoriedad (0=greedy puro, 1=totalmente aleatorio)
    
    Returns:
        tuple: (mejor_secuencia, mejor_costo)
    """
    n = matriz_costos.shape[0]
    mejor_costo = float('inf')
    mejor_secuencia = None

    for _ in range(n_iter):
        secuencia = []
        visitados = set()
        actual = random.randint(0, n - 1)
        secuencia.append(actual)
        visitados.add(actual)

        for _ in range(n - 1):
            candidatos = [(j, matriz_costos[actual, j]) for j in range(n) if j not in visitados]
            if not candidatos:
                break
            candidatos.sort(key=lambda x: x[1])
            rcl_size = max(1, int(len(candidatos) * alpha))
            siguiente, _ = random.choice(candidatos[:rcl_size])
            secuencia.append(siguiente)
            visitados.add(siguiente)
            actual = siguiente

        costo = calcular_costo(matriz_costos, secuencia)
        secuencia, costo = neighbourhood_search(matriz_costos, secuencia)
        
        if costo < mejor_costo:
            mejor_costo = costo
            mejor_secuencia = secuencia[:]
    
    return mejor_secuencia, mejor_costo


In [8]:
def mapear_bks_desde_tours(carpeta_tours):
    """
    Lee los .tour y devuelve un diccionario {ID instancia: BKS} donde el ID es string
    """
    bks_dict = {}
    for archivo in os.listdir(carpeta_tours):
        if archivo.endswith(".tour"):
            base = os.path.splitext(archivo)[0]
            partes = base.split('_')[1].split('.')
            id_instancia = partes[0]
            bks = float(partes[1])
            bks_dict[f"cgl_{id_instancia}.catsp"] = bks
    return bks_dict

## Cargar archivo CATSPP

In [9]:
import os

folder_tsp= os.listdir('data/CGL')
bks= mapear_bks_desde_tours('data/TOURS/CGL')

archivos_tsp = [f for f in folder_tsp if f.endswith('.catsp')]
print(f"Archivos CATSPP encontrados: {archivos_tsp}")

Archivos CATSPP encontrados: ['cgl_66.catsp', 'cgl_38.catsp', 'cgl_60.catsp', 'cgl_51b.catsp', 'cgl_44.catsp', 'cgl_81.catsp', 'cgl_58.catsp', 'cgl_43.catsp', 'cgl_47.catsp', 'cgl_26.catsp', 'cgl_114.catsp', 'cgl_45.catsp', 'cgl_78.catsp', 'cgl_28.catsp', 'cgl_50.catsp', 'cgl_17.catsp', 'cgl_33.catsp', 'cgl_76.catsp', 'cgl_107.catsp', 'cgl_37.catsp', 'cgl_72.catsp', 'cgl_70.catsp', 'cgl_32.catsp', 'cgl_48.catsp', 'cgl_70b.catsp', 'cgl_88.catsp', 'cgl_51.catsp', 'cgl_48b.catsp', 'cgl_73.catsp', 'cgl_57.catsp']


In [10]:
import pandas as pd
import time

results = []

cantidad_archivos = len(archivos_tsp)
count = 0
print(f"Cantidad de archivos CATSPP: {cantidad_archivos}")

for archivo in archivos_tsp:
    count += 1
    print(f"Procesando archivo {count}/{cantidad_archivos}: {archivo}")
    path_archivo = os.path.join('data/CGL', archivo)
    bks_value = bks.get(archivo, None)
    matriz_costos = cargar_instancia_catspp(path_archivo)
    print(f"Matriz de costos cargada: {matriz_costos.shape[0]} x {matriz_costos.shape[1]}")

    start_time_greedy = time.time()
    greedy_secuencia, greedy_costo = greedy_catspp(matriz_costos)
    end_time_greedy = time.time()
    start_time_neighbour = time.time()
    neighbour_secuencia, neighbour_costo = neighbourhood_search(matriz_costos, greedy_secuencia)
    end_time_neighbour = time.time()
    start_time_grasp = time.time()
    grasp_secuencia, grasp_costo = grasp_catspp(matriz_costos)
    end_time_grasp = time.time()

    results.append({
        'archivo': archivo,
        'greedy_secuencia': greedy_secuencia,
        'greedy_costo': greedy_costo,
        'greedy_tiempo': end_time_greedy - start_time_greedy,
        'neighbour_secuencia': neighbour_secuencia,
        'neighbour_costo': neighbour_costo,
        'neighbour_tiempo': end_time_neighbour - start_time_neighbour,
        'grasp_secuencia': grasp_secuencia,
        'grasp_costo': grasp_costo,
        'grasp_tiempo': end_time_grasp - start_time_grasp,
        'bks': bks_value,
        'gap_greedy': greedy_costo - bks_value if bks_value is not None else None,
        'gap_neighbour': neighbour_costo - bks_value if bks_value is not None else None,
        'gap_grasp': grasp_costo - bks_value if bks_value is not None else None
    })


# Convertir resultados a DataFrame
df_results = pd.DataFrame(results)


Cantidad de archivos CATSPP: 30
Procesando archivo 1/30: cgl_66.catsp
Matriz de costos cargada: 67 x 67
Procesando archivo 2/30: cgl_38.catsp
Matriz de costos cargada: 39 x 39
Procesando archivo 3/30: cgl_60.catsp
Matriz de costos cargada: 61 x 61
Procesando archivo 4/30: cgl_51b.catsp
Matriz de costos cargada: 52 x 52
Procesando archivo 5/30: cgl_44.catsp
Matriz de costos cargada: 45 x 45
Procesando archivo 6/30: cgl_81.catsp
Matriz de costos cargada: 82 x 82
Procesando archivo 7/30: cgl_58.catsp
Matriz de costos cargada: 59 x 59
Procesando archivo 8/30: cgl_43.catsp
Matriz de costos cargada: 44 x 44
Procesando archivo 9/30: cgl_47.catsp
Matriz de costos cargada: 48 x 48
Procesando archivo 10/30: cgl_26.catsp
Matriz de costos cargada: 27 x 27
Procesando archivo 11/30: cgl_114.catsp
Matriz de costos cargada: 115 x 115
Procesando archivo 12/30: cgl_45.catsp
Matriz de costos cargada: 46 x 46
Procesando archivo 13/30: cgl_78.catsp
Matriz de costos cargada: 79 x 79
Procesando archivo 14/30

In [ ]:
df_results.to_csv('CATSSP/output/heuristica', index=False)